# Deeper Understanding

When I started the last notebook, I started with the hypothesis that writing code would help.  But there was an implicit assumption in there that the agent *knew what to do*, but was just messing up with the *details of execution*.

It turns out, once I dived deep on each of the 20 sessions, that in fact the biggest problem wasn't that it knew what to do and was having trouble doing it.  Actually what I learned there is that its biggest problem seems to be not actually understanding what the "gist" is of each puzzle.  Its biggest problem is not really deeply and clearly understanding the basic thrust of each of the puzzles.

This problem breaks down into two parts:
1. Figuring out the algorithm, or basic thrust of the problems.
2. Executing the problems once it understands the basic approach.

In this notebook we'll take a shot and seeing if it can even do #2 once it understands the basic thrust.  This is because I may be able to help it understand the basic thrust by attacking item #1, giving it more opportunity to iterate, try different hypotheses, etc.  But if it still can't execute that won't help.